In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
trash = pd.read_csv('trash_hauler_report.csv')
trash.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y
0,25270,11/01/17,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207.0,RED RIVER,3205,2.0,1.727970e+06,686779.478089
1,25274,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218.0,RED RIVER,4202,1.0,1.721259e+06,685444.799565
2,25276,11/01/17,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209.0,RED RIVER,4205,20.0,1.707027e+06,659887.471571
3,25307,11/01/17,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207.0,WASTE IND,2206,2.0,1.735692e+06,685027.245923
4,25312,11/01/17,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209.0,RED RIVER,4203,20.0,1.710186e+06,664205.101066


In [4]:
trash.columns = trash.columns.str.strip()
trash['Zip Code'] = trash['Zip Code'].astype('Int64').astype(str).str.zfill(5)
trash = trash[trash['Request'] != 'Damage to Property']

missed_categories = ['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup']

pattern = r"\bmiss\w*\b|not\s+emptied|not\s+picked\s+up|no\s+pick\s+up" #miss/missed, not emptied, not picked up, no pick up

if isinstance(missed_categories, list):
    missed_pattern = '|'.join(missed_categories)
else:
    missed_pattern = missed_categories 

wmissed = trash[
    (trash['Trash Hauler'].str.contains("Waste IND", case=False, na=False)) &
    (
        (trash['Request'].str.contains(missed_pattern, case=False, na=False)) |
        (trash['Description'].str.contains(pattern, case=False, na=False))
    )
]

wmissed = wmissed.dropna(subset = ['Incident Address']) # drop known missing address

# rrmissed = rrmissed.drop_duplicates(subset=['Incident Address', 'Date Opened']) # I believe this is redundancy and is taken care of later

def normalize_address(s: str) -> str:
    if pd.isna(s):
        return s
    # Only keep everything before the first comma
    s = s.split(",")[0]
    s = s.upper()
    s = re.sub(r"[.,]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    # (keep the direction/street type replacements here)
    return s

wmissed["address_norm"] = wmissed["Incident Address"].apply(normalize_address)

wmissed["Date Opened"] = pd.to_datetime(wmissed["Date Opened"], errors="coerce")
wmissed["date_only"] = wmissed["Date Opened"].dt.date

wmissed_dedup = (
    wmissed.sort_values(['address_norm', 'date_only', 'Date Opened'])
    .drop_duplicates(subset=['address_norm', 'date_only'], keep='first')
    .reset_index(drop=True)
)

gap = wmissed_dedup.groupby("address_norm")["Date Opened"].diff().dt.days
wmissed_dedup["new_event"] = (gap.isna()) | (gap > 6)
wmissed_dedup["event_id"]  = wmissed_dedup.groupby("address_norm")["new_event"].cumsum()

grp = wmissed_dedup.groupby(['address_norm', "event_id"])['Date Opened']
wmissed_dedup["event_date"]     = grp.transform("min")     

wmissed_dedup["event_seq"] = wmissed_dedup["event_id"].astype("Int64")

wmissed_dedup["fine_event"] = (wmissed_dedup["event_seq"] > 1).astype(int) * 200

wmissed_dedup["first_event"] = (wmissed_dedup['Date Opened'] == wmissed_dedup["event_date"])
wmissed_dedup["fine_event"] = wmissed_dedup["fine_event"].where(wmissed_dedup["first_event"], 0)

C:\Users\user\AppData\Local\Temp\ipykernel_35496\2910174628.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  wmissed["Date Opened"] = pd.to_datetime(wmissed["Date Opened"], errors="coerce")


In [5]:
wmissed_dedup

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm,date_only,new_event,event_id,event_date,event_seq,fine_event,first_event
0,207616,2019-06-28,Trash - Curbside/Alley Missed Pickup,Bin is still full at curb as of 12 o?clock noo...,"1000 Lebanon Pike, Nashville, TN 37210, United...",37210,WASTE IND,4507,19.0,1.750261e+06,663845.599995,1000 LEBANON PIKE,2019-06-28,True,1,2019-06-28,1,0,True
1,63499,2018-04-27,Trash - Curbside/Alley Missed Pickup,miss pick up trash,1000 N 14th St,37206,WASTE IND,2506,6.0,1.749176e+06,675585.703139,1000 N 14TH ST,2018-04-27,True,1,2018-04-27,1,0,True
2,126425,2018-12-28,Trash - Backdoor,backdoor/trash miss pick up.,"1002 40th Ave N, Nashville, TN 37209, United S...",37209,WASTE IND,3307,21.0,1.721651e+06,666651.630629,1002 40TH AVE N,2018-12-28,True,1,2018-12-28,1,0,True
3,128686,2019-01-07,Trash - Backdoor,consecutive misses; cust keeps having to call in,"1002 40th Ave N, Nashville, TN 37209, United S...",37209,WASTE IND,3307,21.0,1.721651e+06,666651.630629,1002 40TH AVE N,2019-01-07,True,2,2019-01-07,2,200,True
4,156698,2019-03-20,Trash - Backdoor,still out,"1002 40th Ave N, Nashville, TN 37209, United S...",37209,WASTE IND,3307,21.0,1.721651e+06,666651.630629,1002 40TH AVE N,2019-03-20,True,3,2019-03-20,3,200,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,257713,2019-10-10,Trash - Curbside/Alley Missed Pickup,MISSED THE ENTIRE ALLEY,"945 31st Ave N, Nashville, TN 37209, United St...",37209,WASTE IND,3307,21.0,1.725347e+06,667016.001879,945 31ST AVE N,2019-10-10,True,1,2019-10-10,1,0,True
1132,86128,2018-07-27,Trash - Curbside/Alley Missed Pickup,miss,945 4th Ave S,37210,WASTE IND,4409,17.0,1.741315e+06,661842.669707,945 4TH AVE S,2018-07-27,True,1,2018-07-27,1,0,True
1133,87771,2018-08-03,Trash - Curbside/Alley Missed Pickup,Missed- states they have had to call every wee...,"945 4th Ave S, Nashville, TN 37210, United States",37210,WASTE IND,4409,17.0,1.741251e+06,661765.124996,945 4TH AVE S,2018-08-03,True,2,2018-08-03,2,200,True
1134,88451,2018-08-07,Trash - Curbside/Alley Missed Pickup,This is the second time in about 4 weeks my tr...,951 Sharpe Ave,37206,WASTE IND,2509,5.0,1.747025e+06,676695.302060,951 SHARPE AVE,2018-08-07,True,1,2018-08-07,1,0,True


In [6]:
wmissed_final = wmissed_dedup[wmissed_dedup['new_event'] == True]
wmissed_final

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm,date_only,new_event,event_id,event_date,event_seq,fine_event,first_event
0,207616,2019-06-28,Trash - Curbside/Alley Missed Pickup,Bin is still full at curb as of 12 o?clock noo...,"1000 Lebanon Pike, Nashville, TN 37210, United...",37210,WASTE IND,4507,19.0,1.750261e+06,663845.599995,1000 LEBANON PIKE,2019-06-28,True,1,2019-06-28,1,0,True
1,63499,2018-04-27,Trash - Curbside/Alley Missed Pickup,miss pick up trash,1000 N 14th St,37206,WASTE IND,2506,6.0,1.749176e+06,675585.703139,1000 N 14TH ST,2018-04-27,True,1,2018-04-27,1,0,True
2,126425,2018-12-28,Trash - Backdoor,backdoor/trash miss pick up.,"1002 40th Ave N, Nashville, TN 37209, United S...",37209,WASTE IND,3307,21.0,1.721651e+06,666651.630629,1002 40TH AVE N,2018-12-28,True,1,2018-12-28,1,0,True
3,128686,2019-01-07,Trash - Backdoor,consecutive misses; cust keeps having to call in,"1002 40th Ave N, Nashville, TN 37209, United S...",37209,WASTE IND,3307,21.0,1.721651e+06,666651.630629,1002 40TH AVE N,2019-01-07,True,2,2019-01-07,2,200,True
4,156698,2019-03-20,Trash - Backdoor,still out,"1002 40th Ave N, Nashville, TN 37209, United S...",37209,WASTE IND,3307,21.0,1.721651e+06,666651.630629,1002 40TH AVE N,2019-03-20,True,3,2019-03-20,3,200,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,257713,2019-10-10,Trash - Curbside/Alley Missed Pickup,MISSED THE ENTIRE ALLEY,"945 31st Ave N, Nashville, TN 37209, United St...",37209,WASTE IND,3307,21.0,1.725347e+06,667016.001879,945 31ST AVE N,2019-10-10,True,1,2019-10-10,1,0,True
1132,86128,2018-07-27,Trash - Curbside/Alley Missed Pickup,miss,945 4th Ave S,37210,WASTE IND,4409,17.0,1.741315e+06,661842.669707,945 4TH AVE S,2018-07-27,True,1,2018-07-27,1,0,True
1133,87771,2018-08-03,Trash - Curbside/Alley Missed Pickup,Missed- states they have had to call every wee...,"945 4th Ave S, Nashville, TN 37210, United States",37210,WASTE IND,4409,17.0,1.741251e+06,661765.124996,945 4TH AVE S,2018-08-03,True,2,2018-08-03,2,200,True
1134,88451,2018-08-07,Trash - Curbside/Alley Missed Pickup,This is the second time in about 4 weeks my tr...,951 Sharpe Ave,37206,WASTE IND,2509,5.0,1.747025e+06,676695.302060,951 SHARPE AVE,2018-08-07,True,1,2018-08-07,1,0,True


In [7]:
total_fines = int(wmissed_final["fine_event"].sum())

events_fined = int(
    wmissed_final[wmissed_final['event_id'] != 1].drop_duplicates(['address_norm', "event_id"]).shape[0]
)

In [8]:
total_fines

66800

In [9]:
events_fined

334

In [10]:
wmissed_final.to_csv('wmissed_final.csv', index = False)

In [11]:
wmissed_final_fined = wmissed_final[wmissed_final['fine_event'] != 0]
wmissed_final_fined

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,address_norm,date_only,new_event,event_id,event_date,event_seq,fine_event,first_event
3,128686,2019-01-07,Trash - Backdoor,consecutive misses; cust keeps having to call in,"1002 40th Ave N, Nashville, TN 37209, United S...",37209,WASTE IND,3307,21.0,1.721651e+06,666651.630629,1002 40TH AVE N,2019-01-07,True,2,2019-01-07,2,200,True
4,156698,2019-03-20,Trash - Backdoor,still out,"1002 40th Ave N, Nashville, TN 37209, United S...",37209,WASTE IND,3307,21.0,1.721651e+06,666651.630629,1002 40TH AVE N,2019-03-20,True,3,2019-03-20,3,200,True
22,61779,2018-04-20,Trash - Curbside/Alley Missed Pickup,1011 elm hill pike missed again said if need t...,1011 elm hill pike,37210,WASTE IND,4507,19.0,1.751058e+06,659830.993602,1011 ELM HILL PIKE,2018-04-20,True,2,2018-04-20,2,200,True
23,85157,2018-07-24,Trash - Curbside/Alley Missed Pickup,"Missed on last collection, last Friday out on ...",1011 elm hill pike,37210,WASTE IND,4507,19.0,1.751058e+06,659830.993602,1011 ELM HILL PIKE,2018-07-24,True,3,2018-07-24,3,200,True
24,100002,2018-09-17,Trash - Curbside/Alley Missed Pickup,Missed ..again,1011 elm hill pike,37210,WASTE IND,4507,19.0,1.751060e+06,659827.401172,1011 ELM HILL PIKE,2018-09-17,True,4,2018-09-17,4,200,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,258187,2019-10-11,Trash - Curbside/Alley Missed Pickup,miss,"936 32nd Ave N, Nashville, TN 37209, United St...",37209,WASTE IND,3307,21.0,1.725121e+06,667025.600835,936 32ND AVE N,2019-10-11,True,2,2019-10-11,2,200,True
1120,241803,2019-09-04,Trash - Curbside/Alley Missed Pickup,"Good Afternoon,\r\nWe recently purchased this ...","939 Sharpe Ave, Nashville, TN 37206, United St...",37206,WASTE IND,2509,5.0,1.746761e+06,676724.798403,939 SHARPE AVE,2019-09-04,True,2,2019-09-04,2,200,True
1124,249124,2019-09-19,Trash - Curbside/Alley Missed Pickup,MISSED THE ENTIRE STREET.,940 34th ave n,37209,WASTE IND,3307,21.0,1.724403e+06,666920.801847,940 34TH AVE N,2019-09-19,True,2,2019-09-19,2,200,True
1130,53826,2018-03-15,Trash Collection Complaint,Missed pick up all the time - said no service ...,944 4th Ave S,37210,WASTE IND,4409,17.0,1.741416e+06,661812.758825,944 4TH AVE S,2018-03-15,True,2,2018-03-15,2,200,True


In [12]:
wmissed_final_fined.to_csv('wmissed_final_fined.csv', index = False)

In [27]:
top_zipcodes = wmissed_final['Zip Code'].value_counts()
top_zipcodes

Zip Code
37209    315
37210    220
37208    208
37206    158
37207    102
37203     64
37211      7
0<NA>      5
37219      5
37201      4
37216      1
37213      1
Name: count, dtype: int64

In [37]:
most_frequent_zip = wmissed_final['Zip Code'].value_counts().idxmax()
target_zip = wmissed_final[wmissed_final['Zip Code'] == '37209']
top_zip = len(target_zip)
print(f'The zip code with the most entries is: {most_frequent_zip} with {top_zip} missed pickups')

The zip code with the most entries is: 37209 with 315 missed pickups


In [31]:
top_addresses = wmissed_final['address_norm'].value_counts()
top_addresses

address_norm
15 HERMITAGE AVE        14
123 37TH AVE N           9
1011 ELM HILL PIKE       9
3406 BATAVIA ST          8
518 MONROE ST            7
                        ..
1006 GALLATIN AVE        1
1005 W GREENWOOD AVE     1
1005 SEVIER ST           1
1005 N 14TH ST           1
1005 11TH AVE N          1
Name: count, Length: 756, dtype: int64

In [35]:
most_frequent_address = wmissed_final['address_norm'].value_counts().idxmax()
target_address = wmissed_final[wmissed_final['address_norm'] == '15 HERMITAGE AVE']
top_address = len(target_address)
print(f'The zip code with the most entries is: {most_frequent_address} with {top_address} missed pickups')

The zip code with the most entries is: 15 HERMITAGE AVE with 14 missed pickups
